This notebook is the scratch paper I used while writing [`evaluate.py`](./evaluate.py). I used this whenever I needed an interactive console to try ideas and view results.

## Load and run `evaluate` module

In [1]:
from importlib import reload # recompile module and re-execute module-level code
import evaluate as ev; reload(ev)

<module 'evaluate' from 'D:\\This PC\\Documents\\Github\\nlp-sandbox\\enron\\evaluate.py'>

In [2]:
%run -i evaluate.py

Importing the model from model.pkl
Loading test data
Featurizing test data
Evaluating

Accuracy: 0.3119266055045872

Confusion matrix:
Layout
[[tn   fp]
 [fn   tp]]

[[ 393 1049]
 [   1   83]]

Classification report:
             precision    recall  f1-score   support

        neg       1.00      0.27      0.43      1442
        pos       0.07      0.99      0.14        84

avg / total       0.95      0.31      0.41      1526


Accuracy (on candidates with 1+ feature): 0.6976744186046512

Confusion matrix:
[[393 181]
 [  1  27]]

Classification report:
             precision    recall  f1-score   support

        neg       1.00      0.68      0.81       574
        pos       0.13      0.96      0.23        28

avg / total       0.96      0.70      0.78       602



**Improvement #1**: accuracy filtering above shows the model should learn to be more uncertain about candidates it has zero features on.
   - This could mean not requiring a VERB in order for a candidate to have features. (Although might be reasonable to scope this problem to tasks of this form first, nail it, then expand the definition)
   - This could mean not including training data with 0 features, with the idea that this will force the model to be more uncertain when it comes across 0-feature candidates in test. But is this a bad bias to introduce?

**Improvement #2**: re-train model using spacy v2

I decided to upgrade spacy in the middle of building this notebook (to use the `displacy` visualizer package later on). Went from `1.9.0` to `2.0.7`. Didn't expect it to affect much until I re-train my model. Wrong.

**Measure**|**v1.9.0**|**v2.0.7 (eval-only)**
-----|-----|-----
Accuracy|0.286|0.312
Accuracy on 1+ features|0.633|0.698
Confusion Matrix|[[ 356 1086] [ 4 80]]|[[ 393 1049] [ 1 83]]
Confusion Matrix on 1+ features|[[356 216] [ 4 24]]|[[393 181] [ 1 27]]

So I'm excited to see more performance boosts simply from upgrading Spacy!

_On the negative side, featurization is slower._

Other options after upgrade:
- Add [new morphology info](https://spacy.io/api/annotation#pos-tagging) as features (VerbForm, Tense, Aspect)

**TODO**: address label inbalance

## Using: get_sample_predictions

In [3]:
most_incorrect = ev.get_sample_predictions(preds_df, sample_type=ev.SamplePredictions.MOST_INCORRECT, n=20)
most_incorrect

,candidate_task,prob_dist,label,features,pred,pos_dist,neg_dist,max_dist
396,ENA Origination Support (Getting info from Tyc...,<ProbDist with 2 samples>,neg,"{'VERB': 1, 'FOLLOWING_POS_NOUN': 1, 'FOLLOWIN...",pos,0.973595,0.026405,0.973595
1134,"New York Times on the Web, please contact Alyson",<ProbDist with 2 samples>,neg,"{'VERB': 1, 'FOLLOWING_POS_PROPN': 1, 'FOLLOWI...",pos,0.938270,0.061730,0.938270
1135,Racer at alyson@nytimes.com or visit our onlin...,<ProbDist with 2 samples>,neg,"{'VERB': 1, 'FOLLOWING_POS_ADJ': 1, 'FOLLOWING...",pos,0.933680,0.066320,0.933680
926,wanted only to buy and sell conventional reins...,<ProbDist with 2 samples>,neg,"{'VERB': 3, 'FOLLOWING_POS_ADV': 1, 'FOLLOWING...",pos,0.928333,0.071667,0.928333
33,visit http://www.adamsmark.com/resv/rescheck.h...,<ProbDist with 2 samples>,neg,"{'VERB': 1, 'FOLLOWING_POS_PROPN': 1, 'FOLLOWI...",pos,0.912509,0.087491,0.912509
1328,freeze: bridge,<ProbDist with 2 samples>,neg,"{'VERB': 1, 'FOLLOWING_POS_PUNCT': 1, 'FOLLOWI...",pos,0.911395,0.088605,0.911395
1280,use the following link for enrononline: www.en...,<ProbDist with 2 samples>,neg,"{'VERB': 2, 'FOLLOWING_POS_DET': 1, 'FOLLOWING...",pos,0.908801,0.091199,0.908801
1091,"end, after court hearings on the issue, the Dr...",<ProbDist with 2 samples>,neg,"{'VERB': 1, 'FOLLOWING_POS_PUNCT': 1, 'FOLLOWI...",pos,0.907394,0.092606,0.907394
948,seek to buy replacement coverage to offset dep...,<ProbDist with 2 samples>,neg,"{'VERB': 3, 'FOLLOWING_POS_PART': 1, 'FOLLOWIN...",pos,0.888222,0.111778,0.888222
1137,"For general information about NYTimes.com, wri...",<ProbDist with 2 samples>,neg,"{'VERB': 1, 'FOLLOWING_POS_ADP': 1, 'FOLLOWING...",pos,0.882803,0.117197,0.882803


In [4]:
ev.print_full_df_column(most_incorrect, ['candidate_task', 'label'], True)

candidate_task	label
ENA Origination Support (Getting info from Tycholiz)--$$XXK	neg
New York Times on the Web, please contact Alyson	neg
Racer at alyson@nytimes.com or visit our online media	neg
wanted only to buy and sell conventional reinsurance, Mr. Sweeney	neg
visit http://www.adamsmark.com/resv/rescheck.html.	neg
freeze: bridge	neg
use the following link for enrononline: www.enrononline.com	neg
end, after court hearings on the issue, the Drexel estate	neg
seek to buy replacement coverage to offset depletions in their	neg
For general information about NYTimes.com, write to	neg
the market observations (as opposed to historical data).	neg
execute the agreement and return it to me via fax no. (713) 646-3490.  I will	neg
bring these issues to the table, in a meeting involving all the concerned parties	neg
late Tuesday accused 29 Enron officers and directors of	neg
John Elder, creator of a grass-roots investment program to develop	neg
Get FREE shipping on orders of $75 or more at Starb

### Manual annotations

**candidate\_task**|**label**|**annotation**
:-----:|:-----:|:-----:
ENA Origination Support (Getting info from Tycholiz)--$$XXK|neg|[3(VBG)]
New York Times on the Web, please contact Alyson|neg|[1]
Racer at alyson@nytimes.com or visit our online media|neg|[1]
wanted only to buy and sell conventional reinsurance, Mr. Sweeney|neg|[3(VBD;VB;VB)]
visit http://www.adamsmark.com/resv/rescheck.html.|neg|[1]
freeze: bridge|neg|[1]
use the following link for enrononline: www.enrononline.com|neg|[1]
end, after court hearings on the issue, the Drexel estate|neg|[2]
seek to buy replacement coverage to offset depletions in their|neg|[1][2]
For general information about NYTimes.com, write to|neg|[1][2]
the market observations (as opposed to historical data).|neg|[3(VBN)]
execute the agreement and return it to me via fax no. (713) 646-3490.  I will|neg|[1]
bring these issues to the table, in a meeting involving all the concerned parties|neg|[1]
late Tuesday accused 29 Enron officers and directors of|neg|[2][3(VBD)]
John Elder, creator of a grass-roots investment program to develop|neg|[2][3(VB)]
Get FREE shipping on orders of $75 or more at Starbucks.com|neg|[1]
Managing Director, Enterprise Risk|neg|[3(VBG)]
use the|neg|[1][2]
2. Decision support system: a model recommending transactions to a trader without|neg|[3(VBG)]
convince the external parties (stock analysts, creditors,  credit rating agencies) about|neg|[1]

**Annotation Key**:
- [1] = Reasonable task
- [2] = Sentence fragment
- [3(\*)] = Overweighted VERB (VB\* forms)


([Markdown Table Generator](https://jakebathman.github.io/Markdown-Table-Generator/))

#### Improvements
- **#3** Add feature around the specific kind of VERB (present vs past, etc)
- **#4** Might be useful to classify the greater body of text (e.g., article, paragraph, ?) as something worth further parsing by line - to help avoid sentence fragmentation or pulling "reasonable tasks" out of irrelevant text.

In [5]:
ev.render_displacy(most_incorrect, ev.DisplacyStyles.dep)

['NNP', 'NNP', 'NNP', '-LRB-', 'VBG', 'NN', 'IN', 'NN']
-----



['NNP', 'NNP', 'NNP', 'IN', 'DT', 'NN', ',', 'UH', 'VB', 'NNP']
-----



['NNP', 'IN', 'ADD', 'CC', 'VB', 'PRP$', 'JJ', 'NNS']
-----



['VBD', 'RB', 'TO', 'VB', 'CC', 'VB', 'JJ', 'NN', ',', 'NNP', 'NNP']
-----



['VB', 'NNP', '.']
-----



...

['VB', 'JJ', 'NN', 'IN', 'NNS', 'IN', '$', 'CD', 'CC', 'JJR', 'IN', 'NNP']
-----



['VBG', 'NNP', ',', 'NN', 'NN']
-----



['VB', 'DT']
-----



['LS', '.', 'NN', 'NN', 'NN', ':', 'DT', 'NN', 'VBG', 'NNS', 'IN', 'DT', 'NN', 'IN']
-----



['VB', 'DT', 'JJ', 'NNS', '-LRB-', 'NN', 'NNS', ',', 'NNS', ',', '', 'NN', 'NN', 'NNS', '-RRB-', 'IN']
-----



In [6]:
most_incorrect_pos = ev.get_sample_predictions(preds_df, sample_type=ev.SamplePredictions.MOST_INCORRECT_POS, n=20)
most_incorrect_pos

,candidate_task,prob_dist,label,features,pred,pos_dist,neg_dist,max_dist
248,I need your dues by COB today!,<ProbDist with 2 samples>,pos,"{'VERB': 1, 'FOLLOWING_POS_ADJ': 1, 'FOLLOWING...",neg,0.44212,0.55788,0.55788


**Minor observation**: I've experimented with "commitment" and "request" models trained on email data that would have easily labeled this correctly.

In [7]:
most_correct = ev.get_sample_predictions(preds_df, sample_type=ev.SamplePredictions.MOST_CORRECT, n=20)
most_correct

,candidate_task,prob_dist,label,features,pred,pos_dist,neg_dist,max_dist
83,5. A shark is the only fish that can blink wit...,<ProbDist with 2 samples>,neg,"{'VERB': 2, 'FOLLOWING_POS_DET': 1, 'FOLLOWING...",neg,0.021730,0.978270,0.978270
894,"Currently, the trading that is taking place ty...",<ProbDist with 2 samples>,neg,"{'VERB': 3, 'FOLLOWING_POS_VERB': 1, 'FOLLOWIN...",neg,0.022262,0.977738,0.977738
109,cop and Ernie the taxi driver in Frank Capra's...,<ProbDist with 2 samples>,neg,"{'VERB': 1, 'FOLLOWING_POS_DET': 1, 'FOLLOWING...",neg,0.022798,0.977202,0.977202
612,"The tulips are too excitable, it is winter here.",<ProbDist with 2 samples>,neg,"{'VERB': 2, 'FOLLOWING_POS_ADV': 1, 'FOLLOWING...",neg,0.023710,0.976290,0.976290
1045,Another aspect of this problem is prioritizati...,<ProbDist with 2 samples>,neg,"{'VERB': 1, 'FOLLOWING_POS_NOUN': 1, 'FOLLOWIN...",neg,0.026688,0.973312,0.973312
1001,The most beautiful thing we can experience is ...,<ProbDist with 2 samples>,neg,"{'VERB': 3, 'FOLLOWING_POS_VERB': 1, 'FOLLOWIN...",neg,0.029022,0.970978,0.970978
121,"35. ""Stewardesses"" is the longest word that is...",<ProbDist with 2 samples>,neg,"{'VERB': 3, 'FOLLOWING_POS_DET': 1, 'FOLLOWING...",neg,0.030214,0.969786,0.969786
1027,trading desks is perfect. One problem that is ...,<ProbDist with 2 samples>,neg,"{'VERB': 3, 'FOLLOWING_POS_ADJ': 2, 'FOLLOWING...",neg,0.033225,0.966775,0.966775
1000,Imagination is more important than knowledge...,<ProbDist with 2 samples>,neg,"{'VERB': 1, 'FOLLOWING_POS_ADV': 1, 'FOLLOWING...",neg,0.034947,0.965053,0.965053
1032,One way to address the problem is to implement...,<ProbDist with 2 samples>,neg,"{'VERB': 4, 'FOLLOWING_POS_DET': 2, 'FOLLOWING...",neg,0.035199,0.964801,0.964801


In [8]:
ev.print_full_df_column(most_correct, ['candidate_task', 'label'], True)

candidate_task	label
5. A shark is the only fish that can blink with both eyes.	neg
Currently, the trading that is taking place typically involves	neg
"cop and Ernie the taxi driver in Frank Capra's ""It's a Wonderful Life."""	neg
The tulips are too excitable, it is winter here.	neg
Another aspect of this problem is prioritization of different projects. In my view	neg
"The most beautiful thing we can experience is the mysterious.  It is	neg
the source of all true art and science."	neg
"35. ""Stewardesses"" is the longest word that is typed with only the left"	neg
trading desks is perfect. One problem that is persistent in some portfolios is the disconnect	neg
Imagination is more important than knowledge...	neg
One way to address the problem is to implement a module that compares systematically	neg
What is missing is the industrial type process that guarantees the quality	neg
building is an American flag.	neg
The old man steps up to the tee and hits the ball. It goes sailing	neg
2. I ha

In [9]:
most_correct_pos = ev.get_sample_predictions(preds_df, sample_type=ev.SamplePredictions.MOST_CORRECT_POS, n=20)
most_correct_pos

,candidate_task,prob_dist,label,features,pred,pos_dist,neg_dist,max_dist
1517,"install v4.0 build 23, follow old instructions...",<ProbDist with 2 samples>,pos,"{'VERB': 4, 'FOLLOWING_POS_NOUN': 3, 'FOLLOWIN...",pos,0.937159,0.062841,0.937159
273,Check last years tax return for RRSP contribut...,<ProbDist with 2 samples>,pos,"{'VERB': 1, 'FOLLOWING_POS_ADJ': 1, 'FOLLOWING...",pos,0.923122,0.076878,0.923122
1255,Call Ambrosia ? Monday morning,<ProbDist with 2 samples>,pos,"{'VERB': 1, 'FOLLOWING_POS_PROPN': 1, 'FOLLOWI...",pos,0.912509,0.087491,0.912509
605,Send ED-mail to Ed.,<ProbDist with 2 samples>,pos,"{'VERB': 1, 'FOLLOWING_POS_NOUN': 1, 'FOLLOWIN...",pos,0.910623,0.089377,0.910623
163,Check for ML,<ProbDist with 2 samples>,pos,"{'VERB': 1, 'FOLLOWING_POS_ADP': 1, 'FOLLOWING...",pos,0.905871,0.094129,0.905871
127,Set up a folder on the C-drive called WINZIP a...,<ProbDist with 2 samples>,pos,"{'VERB': 4, 'FOLLOWING_POS_PART': 1, 'FOLLOWIN...",pos,0.904842,0.095158,0.904842
603,Send E-mail to Ed.,<ProbDist with 2 samples>,pos,"{'VERB': 1, 'FOLLOWING_POS_NOUN': 1, 'FOLLOWIN...",pos,0.902484,0.097516,0.902484
1516,please download and make a backup copy.,<ProbDist with 2 samples>,pos,"{'VERB': 2, 'FOLLOWING_POS_CCONJ': 1, 'FOLLOWI...",pos,0.899506,0.100494,0.899506
1215,Call Grant about Tony,<ProbDist with 2 samples>,pos,"{'VERB': 1, 'FOLLOWING_POS_PROPN': 1, 'FOLLOWI...",pos,0.881632,0.118368,0.881632
75,resend pheasant email to all.df,<ProbDist with 2 samples>,pos,"{'VERB': 2, 'FOLLOWING_POS_ADJ': 1, 'FOLLOWING...",pos,0.874347,0.125653,0.874347


In [10]:
most_uncertain = ev.get_sample_predictions(preds_df, sample_type=ev.SamplePredictions.MOST_UNCERTAIN, n=20)
most_uncertain

,candidate_task,prob_dist,label,features,pred,pos_dist,neg_dist,max_dist
648,Even through the gift paper I could hear them ...,<ProbDist with 2 samples>,neg,"{'VERB': 2, 'FOLLOWING_POS_PRON': 1, 'FOLLOWIN...",neg,0.499242,0.500758,0.500758
727,very interested in this guy and would be ready...,<ProbDist with 2 samples>,neg,"{'VERB': 2, 'FOLLOWING_POS_ADJ': 1, 'FOLLOWING...",pos,0.500941,0.499059,0.500941
1108,"to certain employees, and not proceed with the...",<ProbDist with 2 samples>,neg,"{'VERB': 1, 'FOLLOWING_POS_ADP': 1, 'FOLLOWING...",pos,0.502288,0.497712,0.502288
1097,up as to whether there was fair consideration ...,<ProbDist with 2 samples>,neg,"{'VERB': 2, 'FOLLOWING_POS_ADJ': 1, 'FOLLOWING...",pos,0.502583,0.497417,0.502583
559,Freeman indicated that the Power Authority has...,<ProbDist with 2 samples>,neg,"{'VERB': 5, 'FOLLOWING_POS_ADP': 2, 'FOLLOWING...",pos,0.502932,0.497068,0.502932
786,"For your convenience, this workshop will be he...",<ProbDist with 2 samples>,neg,"{'VERB': 2, 'FOLLOWING_POS_VERB': 1, 'FOLLOWIN...",neg,0.496220,0.503780,0.503780
998,You cannot simultaneously prevent and prepare ...,<ProbDist with 2 samples>,neg,"{'VERB': 2, 'FOLLOWING_POS_CCONJ': 1, 'FOLLOWI...",neg,0.495866,0.504134,0.504134
903,"of Trade, which began trading the options in 1...",<ProbDist with 2 samples>,neg,"{'VERB': 2, 'FOLLOWING_POS_VERB': 1, 'FOLLOWIN...",neg,0.493854,0.506146,0.506146
380,maine impossible to get to...next option?,<ProbDist with 2 samples>,pos,"{'VERB': 1, 'FOLLOWING_POS_ADP': 1, 'FOLLOWING...",pos,0.510187,0.489813,0.510187
632,"Their smiles catch onto my skin, little smilin...",<ProbDist with 2 samples>,neg,"{'VERB': 2, 'FOLLOWING_POS_ADP': 1, 'FOLLOWING...",pos,0.510244,0.489756,0.510244


In [11]:
ev.print_full_df_column(most_uncertain, ['candidate_task', 'pred', 'label'], True)

candidate_task	pred	label
Even through the gift paper I could hear them breathe	neg	neg
very interested in this guy and would be ready to bring him over	pos	neg
to certain employees, and not proceed with the payments	pos	neg
up as to whether there was fair consideration given for the	pos	neg
Freeman indicated that the Power Authority has signed letters of intent to purchase output from 14 biomass facilities in the Central Valley, as well as 400 MW generated by wind.	pos	neg
For your convenience, this workshop will be held at three different times	neg	neg
You cannot simultaneously prevent and prepare for war.	neg	neg
of Trade, which began trading the options in 1996.	neg	neg
maine impossible to get to...next option?	pos	pos
Their smiles catch onto my skin, little smiling hooks.	pos	neg
paste into this e-mail system it happens automatically.	pos	neg
a mild winter, and it would be able to use the futures to hedge a	neg	neg
Keeping it all together, through the change.	neg	neg
tmpl_name=sto

In [12]:
ev.confusion_matrix(most_uncertain[['candidate_task', 'prob_dist', 'label', 'features']].itertuples(index=False))

array([[11,  8],
       [ 0,  1]])

#### Q: What TP candidates (actual tasks) didn't have features?

In [13]:
all_correct_pos = ev.get_sample_predictions(preds_df, sample_type=ev.SamplePredictions.MOST_CORRECT_POS)

In [14]:
len(all_correct_pos)

83

In [15]:
no_features_mask = all_correct_pos['features'] == {}
tp_no_features = all_correct_pos.iloc[np.where(no_features_mask)]
tp_no_features

,candidate_task,prob_dist,label,features,pred,pos_dist,neg_dist,max_dist
596,Vince,<ProbDist with 2 samples>,pos,{},pos,0.560698,0.439302,0.560698
527,1. M. Soto,<ProbDist with 2 samples>,pos,{},pos,0.560698,0.439302,0.560698
499,1. Ecopetrol,<ProbDist with 2 samples>,pos,{},pos,0.560698,0.439302,0.560698
597,Articles,<ProbDist with 2 samples>,pos,{},pos,0.560698,0.439302,0.560698
602,"Dentist on Jan 25, 7:45.",<ProbDist with 2 samples>,pos,{},pos,0.560698,0.439302,0.560698
604,"Dentist on Jan 25, 7:45.",<ProbDist with 2 samples>,pos,{},pos,0.560698,0.439302,0.560698
1171,EES presentations to Rhys,<ProbDist with 2 samples>,pos,{},pos,0.560698,0.439302,0.560698
746,McCormack,<ProbDist with 2 samples>,pos,{},pos,0.560698,0.439302,0.560698
747,Article about the group,<ProbDist with 2 samples>,pos,{},pos,0.560698,0.439302,0.560698
753,"1, Li Xiao",<ProbDist with 2 samples>,pos,{},pos,0.560698,0.439302,0.560698


**Improvement #5**: Manual re-labeling due to my "split by new line" decision is necessary for more accurate results. All sample datasets have instances of mis-labeled data.

In [16]:
ev.render_displacy(tp_no_features, [ev.DisplacyStyles.dep, ev.DisplacyStyles.ent])

['NN']
-----



['LS', '.', 'NNP', 'NNP']
-----



['LS', '.', 'NN']
-----



['NNS']
-----



['NN', 'IN', '', 'NNP', 'CD', ',', 'CD', '.']
-----



...

['CD', 'SYM', 'CD']
-----



['NNP', 'NNP']
-----



['NN', 'NN']
-----



['NN', 'CD', 'NNP', 'NN', 'IN', 'NNP']
-----



['NNP']
-----



### Observations

- Model is bad at:
    - Incomplete sentences/fragments
    - Tasks without verbs (noun phrases)
    - Taking verb tense into account
- Model is good at:
    - Finding tasks in weird contexts
- Model is most uncertain about: 

## Writing: get_sample_predictions

Understanding code from [fast.ai 2018: lesson 1](https://github.com/fastai/fastai/blob/master/courses/dl1/lesson1.ipynb) for getting various types of sample predictions.

`def rand_by_mask(mask, n): return np.random.choice(np.where(mask)[0], n, replace=False)`

    np.random.choice(a, size=None, replace=True, p=None)

    np.where(condition[, x, y])

`def rand_by_correct(preds, is_correct): return rand_by_mask((preds.preds == preds.labels)==is_correct)`
    
    np.where((preds.preds == preds.labels)==is_correct))

Goals:
1. Get `preds` into a form so that `rand_by_mask` works on it
2. Get `preds` into a form so that we can find what is correct vs incorrect

### What is `preds`?

In [17]:
type(preds)

list

In [18]:
type(preds[1]), preds[1]

(tuple,
 ('http://gasfundy.corp.enron.com/gas/framework/default.asp',
  <ProbDist with 2 samples>,
  'neg',
  {}))

where `tuple == (candidate_task, prediction_dist, label, features)`

In [19]:
[(p[0], p[1].max(), p[2], p[3]) for p in preds][1]

('http://gasfundy.corp.enron.com/gas/framework/default.asp', 'pos', 'neg', {})

In [20]:
type(preds[1][1]), preds[1][1]

(nltk.probability.DictionaryProbDist, <ProbDist with 2 samples>)

In [21]:
import nltk
??nltk.probability.DictionaryProbDist

In [22]:
preds[1][1].samples()

dict_keys(['neg', 'pos'])

In [23]:
preds[1][1].max()

'pos'

In [24]:
preds[1][1].prob('pos'), preds[1][1].prob('neg')

(0.56069809800114434, 0.43930190199885577)

### Form of `preds` that works with `rand_by_mask(mask, n)`?

For `np.random.choice`, first arg must be 1-dimensional

In [25]:
import numpy as np
np_preds = np.array(preds)
np_preds.shape

(1526, 4)

`preds` as it stands will not work. Instead, we can feed in the length of `preds` and let `np.random.choice` select random indices of `preds`.

In [26]:
n = 5 # number of results
mask = len(np_preds)

In [27]:
np_random = np_preds[np.random.choice(mask, n)]

In [28]:
np_random

array([['http://www.science-finance.fr/publications.html',
        <ProbDist with 2 samples>, 'neg', {}],
       ['They pass the way gulls pass inland in their white caps,',
        <ProbDist with 2 samples>, 'neg',
        {'VERB': 2, 'FOLLOWING_POS_DET': 1, 'FOLLOWING_POSTAG_DT': 1, 'PARENT_DEP_ROOT': 1, 'PARENT_POS_VERB': 1, 'PARENT_POSTAG_VBP': 1, 'CHILD_DEP_NSUBJ': 2, 'CHILD_POS_PRON': 1, 'CHILD_POSTAG_PRP': 1, 'CHILD_DEP_DOBJ': 1, 'CHILD_POS_NOUN': 2, 'CHILD_POSTAG_NN': 1, 'CHILD_DEP_PUNCT': 1, 'CHILD_POS_PUNCT': 1, 'CHILD_POSTAG_,': 1, 'FOLLOWING_POS_ADV': 1, 'FOLLOWING_POSTAG_RB': 1, 'PARENT_DEP_DOBJ': 1, 'PARENT_POS_NOUN': 1, 'PARENT_POSTAG_NN': 1, 'CHILD_POSTAG_NNS': 1, 'CHILD_DEP_ADVMOD': 1, 'CHILD_POS_ADV': 1, 'CHILD_POSTAG_RB': 1, 'CHILD_DEP_PREP': 1, 'CHILD_POS_ADP': 1, 'CHILD_POSTAG_IN': 1}],
       ['Houston Chronicle Online', <ProbDist with 2 samples>, 'neg', {}],
       ['ID:         mcconnellmarkr', <ProbDist with 2 samples>, 'neg', {}],
       ['https://www.admissio

In [29]:
len(np_random)

5

### Form of `preds` that works with `rand_by_correct(preds, is_correct)`?

`preds` must be broken down into predictions and labels. So first, let's get the predictions.

Remember that each `tuple` in `preds` == `(candidate_task, prediction_dist, label, features)`

In [30]:
[p[1].max() for p in preds][:n]

['pos', 'pos', 'pos', 'pos', 'pos']

Labels can be retrieved similarly

In [31]:
[p[2] for p in preds][:n]

['neg', 'neg', 'neg', 'neg', 'neg']

Noticing that `preds` might be more readable as a pandas dataframe. So let's convert it.

In [32]:
import pandas as pd

preds_df = pd.DataFrame.from_records(preds, columns=['candidate_task', 'prob_dist', 'label', 'features'])
preds_df['pred'] = [p.max() for p in preds_df['prob_dist']]

In [33]:
preds_df[:n]

,candidate_task,prob_dist,label,features,pred
0,https://www4.rsweb.com/61045/,<ProbDist with 2 samples>,neg,{},pos
1,http://gasfundy.corp.enron.com/gas/framework/d...,<ProbDist with 2 samples>,neg,{},pos
2,212 836 5030,<ProbDist with 2 samples>,neg,{},pos
3,http://fundamentals.corp.enron.com/main.asp,<ProbDist with 2 samples>,neg,{},pos
4,adams 30 631,<ProbDist with 2 samples>,neg,"{'VERB': 1, 'FOLLOWING_POS_NUM': 1, 'FOLLOWING...",pos


In [34]:
[p.max() for p in preds_df['prob_dist']][:n]

['pos', 'pos', 'pos', 'pos', 'pos']

In [35]:
preds_df['label'][:n].values

array(['neg', 'neg', 'neg', 'neg', 'neg'], dtype=object)

Put this all together to write our own 

#### `rand_by_correct(preds, n=5, is_correct=True)`

In [36]:
def rand_by_mask(mask, n):
    return np.random.choice(np.where(mask)[0], n, replace=False)

def rand_by_correct(preds_df, n = 5, is_correct = True):
    return preds_df.iloc[rand_by_mask(([p.max() for p in preds_df['prob_dist']] == preds_df['label'].values)==is_correct, n)]

In [37]:
# correct
rand_by_correct(preds_df)

,candidate_task,prob_dist,label,features,pred
1294,There were reports of further attacks overnigh...,<ProbDist with 2 samples>,neg,"{'VERB': 4, 'FOLLOWING_POS_NOUN': 2, 'FOLLOWIN...",neg
987,As far as the laws of mathematics refer to rea...,<ProbDist with 2 samples>,neg,"{'VERB': 5, 'FOLLOWING_POS_ADP': 2, 'FOLLOWING...",neg
890,to the mortgage-backed securities that have be...,<ProbDist with 2 samples>,neg,"{'VERB': 3, 'FOLLOWING_POS_NOUN': 1, 'FOLLOWIN...",neg
1409,"Fuel transactions include fuel oil, and I thou...",<ProbDist with 2 samples>,neg,"{'VERB': 4, 'FOLLOWING_POS_NOUN': 2, 'FOLLOWIN...",neg
1030,forward price volati;oty. If this is not the c...,<ProbDist with 2 samples>,neg,"{'VERB': 1, 'FOLLOWING_POS_ADV': 1, 'FOLLOWING...",neg


In [38]:
# incorrect
rand_by_correct(preds_df, is_correct=False)

,candidate_task,prob_dist,label,features,pred
1331,gregory,<ProbDist with 2 samples>,neg,{},pos
429,Oct-03,<ProbDist with 2 samples>,neg,{},pos
20,----------------------------------------------...,<ProbDist with 2 samples>,neg,{},pos
1374,6 mnth 575 15-20% discount,<ProbDist with 2 samples>,neg,{},pos
1156,GemanIX@aol.com,<ProbDist with 2 samples>,neg,{},pos


Cool! Adding more goals... this time around getting the 'most correct/incorrect'

`def most_by_mask(mask, mult):
    idxs = np.where(mask)[0]
    return idxs[np.argsort(mult * probs[idxs])[:4]]`
    
    numpy.argsort(a, axis=-1, kind='quicksort', order=None)

`def most_by_correct(y, is_correct): 
    mult = -1 if (y==1)==is_correct else 1
    return most_by_mask(((preds == data.val_y)==is_correct) & (data.val_y == y), mult)`
    
    mult: makes it so that if 'correct' == 'lowest probabilities', the argsort sees lower == more correct. else, higher == more correct

Goals:
1. Get `preds` into a form so that `most_by_mask` works on it
2. Get `preds` into a form so that we can find what is _most_ correct vs _most_ incorrect

Unlike the `Cats vs Dogs` predictions, NLTK provides separate probability distributions for each possible label.

Sorting is easy if we break up the values within the `prob_dist` column

In [39]:
[p.prob('pos') for p in preds_df['prob_dist']][:n]

[0.56069809800114434,
 0.56069809800114434,
 0.56069809800114434,
 0.56069809800114434,
 0.59312608626826946]

In [40]:
[p.prob('neg') for p in preds_df['prob_dist']][:n]

[0.43930190199885577,
 0.43930190199885577,
 0.43930190199885577,
 0.43930190199885577,
 0.40687391373173049]

In [41]:
preds_df['pos_dist'] = [p.prob('pos') for p in preds_df['prob_dist']]
preds_df['neg_dist'] = [p.prob('neg') for p in preds_df['prob_dist']]

In [42]:
preds_df[:n]

,candidate_task,prob_dist,label,features,pred,pos_dist,neg_dist
0,https://www4.rsweb.com/61045/,<ProbDist with 2 samples>,neg,{},pos,0.560698,0.439302
1,http://gasfundy.corp.enron.com/gas/framework/d...,<ProbDist with 2 samples>,neg,{},pos,0.560698,0.439302
2,212 836 5030,<ProbDist with 2 samples>,neg,{},pos,0.560698,0.439302
3,http://fundamentals.corp.enron.com/main.asp,<ProbDist with 2 samples>,neg,{},pos,0.560698,0.439302
4,adams 30 631,<ProbDist with 2 samples>,neg,"{'VERB': 1, 'FOLLOWING_POS_NUM': 1, 'FOLLOWING...",pos,0.593126,0.406874


In [43]:
preds_df.sort_values('pos_dist', ascending=False)[:5]

,candidate_task,prob_dist,label,features,pred,pos_dist,neg_dist
396,ENA Origination Support (Getting info from Tyc...,<ProbDist with 2 samples>,neg,"{'VERB': 1, 'FOLLOWING_POS_NOUN': 1, 'FOLLOWIN...",pos,0.973595,0.026405
1134,"New York Times on the Web, please contact Alyson",<ProbDist with 2 samples>,neg,"{'VERB': 1, 'FOLLOWING_POS_PROPN': 1, 'FOLLOWI...",pos,0.938270,0.061730
1517,"install v4.0 build 23, follow old instructions...",<ProbDist with 2 samples>,pos,"{'VERB': 4, 'FOLLOWING_POS_NOUN': 3, 'FOLLOWIN...",pos,0.937159,0.062841
1135,Racer at alyson@nytimes.com or visit our onlin...,<ProbDist with 2 samples>,neg,"{'VERB': 1, 'FOLLOWING_POS_ADJ': 1, 'FOLLOWING...",pos,0.933680,0.066320
926,wanted only to buy and sell conventional reins...,<ProbDist with 2 samples>,neg,"{'VERB': 3, 'FOLLOWING_POS_ADV': 1, 'FOLLOWING...",pos,0.928333,0.071667


In [44]:
preds_df.sort_values('neg_dist', ascending=False)[:5]

,candidate_task,prob_dist,label,features,pred,pos_dist,neg_dist
83,5. A shark is the only fish that can blink wit...,<ProbDist with 2 samples>,neg,"{'VERB': 2, 'FOLLOWING_POS_DET': 1, 'FOLLOWING...",neg,0.021730,0.978270
894,"Currently, the trading that is taking place ty...",<ProbDist with 2 samples>,neg,"{'VERB': 3, 'FOLLOWING_POS_VERB': 1, 'FOLLOWIN...",neg,0.022262,0.977738
109,cop and Ernie the taxi driver in Frank Capra's...,<ProbDist with 2 samples>,neg,"{'VERB': 1, 'FOLLOWING_POS_DET': 1, 'FOLLOWING...",neg,0.022798,0.977202
612,"The tulips are too excitable, it is winter here.",<ProbDist with 2 samples>,neg,"{'VERB': 2, 'FOLLOWING_POS_ADV': 1, 'FOLLOWING...",neg,0.023710,0.976290
1045,Another aspect of this problem is prioritizati...,<ProbDist with 2 samples>,neg,"{'VERB': 1, 'FOLLOWING_POS_NOUN': 1, 'FOLLOWIN...",neg,0.026688,0.973312


In [45]:
preds[1][1].samples()

dict_keys(['neg', 'pos'])

Drumroll...

#### most_by_correct(preds_df, label, n=5, is_correct=True)

In [46]:
def most_by_correct(preds_df, label, n=5, is_correct=True):
    label_options = list(preds_df['prob_dist'][0].samples())
    # note the 2-class assumption
    other_label = label_options[abs(label_options.index(label)-1)]
    # if is_correct, label dist is what matters; if incorrect, look at other_label dist
    dist_column = label + '_dist' if is_correct == True else other_label + '_dist'
    
    correct_mask = ([p.max() for p in preds_df['prob_dist']] == preds_df['label'].values)==is_correct
    target_label_mask = preds_df['label'].values == label
    
    return preds_df.iloc[np.where(correct_mask & target_label_mask)].sort_values(dist_column, ascending=False)[:n]

In [47]:
most_by_correct(preds_df, 'pos')

,candidate_task,prob_dist,label,features,pred,pos_dist,neg_dist
1517,"install v4.0 build 23, follow old instructions...",<ProbDist with 2 samples>,pos,"{'VERB': 4, 'FOLLOWING_POS_NOUN': 3, 'FOLLOWIN...",pos,0.937159,0.062841
273,Check last years tax return for RRSP contribut...,<ProbDist with 2 samples>,pos,"{'VERB': 1, 'FOLLOWING_POS_ADJ': 1, 'FOLLOWING...",pos,0.923122,0.076878
1255,Call Ambrosia ? Monday morning,<ProbDist with 2 samples>,pos,"{'VERB': 1, 'FOLLOWING_POS_PROPN': 1, 'FOLLOWI...",pos,0.912509,0.087491
605,Send ED-mail to Ed.,<ProbDist with 2 samples>,pos,"{'VERB': 1, 'FOLLOWING_POS_NOUN': 1, 'FOLLOWIN...",pos,0.910623,0.089377
163,Check for ML,<ProbDist with 2 samples>,pos,"{'VERB': 1, 'FOLLOWING_POS_ADP': 1, 'FOLLOWING...",pos,0.905871,0.094129


In [48]:
most_by_correct(preds_df, 'neg')

,candidate_task,prob_dist,label,features,pred,pos_dist,neg_dist
83,5. A shark is the only fish that can blink wit...,<ProbDist with 2 samples>,neg,"{'VERB': 2, 'FOLLOWING_POS_DET': 1, 'FOLLOWING...",neg,0.021730,0.978270
894,"Currently, the trading that is taking place ty...",<ProbDist with 2 samples>,neg,"{'VERB': 3, 'FOLLOWING_POS_VERB': 1, 'FOLLOWIN...",neg,0.022262,0.977738
109,cop and Ernie the taxi driver in Frank Capra's...,<ProbDist with 2 samples>,neg,"{'VERB': 1, 'FOLLOWING_POS_DET': 1, 'FOLLOWING...",neg,0.022798,0.977202
612,"The tulips are too excitable, it is winter here.",<ProbDist with 2 samples>,neg,"{'VERB': 2, 'FOLLOWING_POS_ADV': 1, 'FOLLOWING...",neg,0.023710,0.976290
1045,Another aspect of this problem is prioritizati...,<ProbDist with 2 samples>,neg,"{'VERB': 1, 'FOLLOWING_POS_NOUN': 1, 'FOLLOWIN...",neg,0.026688,0.973312


In [49]:
most_by_correct(preds_df, 'pos', is_correct=False)

,candidate_task,prob_dist,label,features,pred,pos_dist,neg_dist
248,I need your dues by COB today!,<ProbDist with 2 samples>,pos,"{'VERB': 1, 'FOLLOWING_POS_ADJ': 1, 'FOLLOWING...",neg,0.44212,0.55788


In [50]:
most_by_correct(preds_df, 'neg', is_correct=False)

,candidate_task,prob_dist,label,features,pred,pos_dist,neg_dist
396,ENA Origination Support (Getting info from Tyc...,<ProbDist with 2 samples>,neg,"{'VERB': 1, 'FOLLOWING_POS_NOUN': 1, 'FOLLOWIN...",pos,0.973595,0.026405
1134,"New York Times on the Web, please contact Alyson",<ProbDist with 2 samples>,neg,"{'VERB': 1, 'FOLLOWING_POS_PROPN': 1, 'FOLLOWI...",pos,0.938270,0.061730
1135,Racer at alyson@nytimes.com or visit our onlin...,<ProbDist with 2 samples>,neg,"{'VERB': 1, 'FOLLOWING_POS_ADJ': 1, 'FOLLOWING...",pos,0.933680,0.066320
926,wanted only to buy and sell conventional reins...,<ProbDist with 2 samples>,neg,"{'VERB': 3, 'FOLLOWING_POS_ADV': 1, 'FOLLOWING...",pos,0.928333,0.071667
33,visit http://www.adamsmark.com/resv/rescheck.h...,<ProbDist with 2 samples>,neg,"{'VERB': 1, 'FOLLOWING_POS_PROPN': 1, 'FOLLOWI...",pos,0.912509,0.087491


Final `get_sample_predictions` goal: get most uncertain

`most_uncertain = np.argsort(np.abs(probs -0.5))[:4]`

Basically, we're going to sort by probabilities closest to 0.5 (on either side)

#### most_uncertain(preds_df, label, n=5)

In [51]:
def most_uncertain(preds_df, label, n=5):
    dist_column = label + '_dist'
    return preds_df.iloc[np.argsort(np.abs(preds_df[dist_column]-0.5))][:n]

In [52]:
most_uncertain(preds_df, 'pos', 10)

,candidate_task,prob_dist,label,features,pred,pos_dist,neg_dist
648,Even through the gift paper I could hear them ...,<ProbDist with 2 samples>,neg,"{'VERB': 2, 'FOLLOWING_POS_PRON': 1, 'FOLLOWIN...",neg,0.499242,0.500758
727,very interested in this guy and would be ready...,<ProbDist with 2 samples>,neg,"{'VERB': 2, 'FOLLOWING_POS_ADJ': 1, 'FOLLOWING...",pos,0.500941,0.499059
1108,"to certain employees, and not proceed with the...",<ProbDist with 2 samples>,neg,"{'VERB': 1, 'FOLLOWING_POS_ADP': 1, 'FOLLOWING...",pos,0.502288,0.497712
1097,up as to whether there was fair consideration ...,<ProbDist with 2 samples>,neg,"{'VERB': 2, 'FOLLOWING_POS_ADJ': 1, 'FOLLOWING...",pos,0.502583,0.497417
559,Freeman indicated that the Power Authority has...,<ProbDist with 2 samples>,neg,"{'VERB': 5, 'FOLLOWING_POS_ADP': 2, 'FOLLOWING...",pos,0.502932,0.497068
786,"For your convenience, this workshop will be he...",<ProbDist with 2 samples>,neg,"{'VERB': 2, 'FOLLOWING_POS_VERB': 1, 'FOLLOWIN...",neg,0.496220,0.503780
998,You cannot simultaneously prevent and prepare ...,<ProbDist with 2 samples>,neg,"{'VERB': 2, 'FOLLOWING_POS_CCONJ': 1, 'FOLLOWI...",neg,0.495866,0.504134
903,"of Trade, which began trading the options in 1...",<ProbDist with 2 samples>,neg,"{'VERB': 2, 'FOLLOWING_POS_VERB': 1, 'FOLLOWIN...",neg,0.493854,0.506146
380,maine impossible to get to...next option?,<ProbDist with 2 samples>,pos,"{'VERB': 1, 'FOLLOWING_POS_ADP': 1, 'FOLLOWING...",pos,0.510187,0.489813
632,"Their smiles catch onto my skin, little smilin...",<ProbDist with 2 samples>,neg,"{'VERB': 2, 'FOLLOWING_POS_ADP': 1, 'FOLLOWING...",pos,0.510244,0.489756


In [53]:
most_uncertain(preds_df, 'neg', 10)

,candidate_task,prob_dist,label,features,pred,pos_dist,neg_dist
648,Even through the gift paper I could hear them ...,<ProbDist with 2 samples>,neg,"{'VERB': 2, 'FOLLOWING_POS_PRON': 1, 'FOLLOWIN...",neg,0.499242,0.500758
727,very interested in this guy and would be ready...,<ProbDist with 2 samples>,neg,"{'VERB': 2, 'FOLLOWING_POS_ADJ': 1, 'FOLLOWING...",pos,0.500941,0.499059
1108,"to certain employees, and not proceed with the...",<ProbDist with 2 samples>,neg,"{'VERB': 1, 'FOLLOWING_POS_ADP': 1, 'FOLLOWING...",pos,0.502288,0.497712
1097,up as to whether there was fair consideration ...,<ProbDist with 2 samples>,neg,"{'VERB': 2, 'FOLLOWING_POS_ADJ': 1, 'FOLLOWING...",pos,0.502583,0.497417
559,Freeman indicated that the Power Authority has...,<ProbDist with 2 samples>,neg,"{'VERB': 5, 'FOLLOWING_POS_ADP': 2, 'FOLLOWING...",pos,0.502932,0.497068
786,"For your convenience, this workshop will be he...",<ProbDist with 2 samples>,neg,"{'VERB': 2, 'FOLLOWING_POS_VERB': 1, 'FOLLOWIN...",neg,0.496220,0.503780
998,You cannot simultaneously prevent and prepare ...,<ProbDist with 2 samples>,neg,"{'VERB': 2, 'FOLLOWING_POS_CCONJ': 1, 'FOLLOWI...",neg,0.495866,0.504134
903,"of Trade, which began trading the options in 1...",<ProbDist with 2 samples>,neg,"{'VERB': 2, 'FOLLOWING_POS_VERB': 1, 'FOLLOWIN...",neg,0.493854,0.506146
380,maine impossible to get to...next option?,<ProbDist with 2 samples>,pos,"{'VERB': 1, 'FOLLOWING_POS_ADP': 1, 'FOLLOWING...",pos,0.510187,0.489813
632,"Their smiles catch onto my skin, little smilin...",<ProbDist with 2 samples>,neg,"{'VERB': 2, 'FOLLOWING_POS_ADP': 1, 'FOLLOWING...",pos,0.510244,0.489756


Turns out, both `label`s return the same list

## Writing: aggregate_features

how to correlate features to (in)correctness

Example set of feature dictionaries...

In [54]:
most_incorrect[['features']]

,features
396,"{'VERB': 1, 'FOLLOWING_POS_NOUN': 1, 'FOLLOWIN..."
1134,"{'VERB': 1, 'FOLLOWING_POS_PROPN': 1, 'FOLLOWI..."
1135,"{'VERB': 1, 'FOLLOWING_POS_ADJ': 1, 'FOLLOWING..."
926,"{'VERB': 3, 'FOLLOWING_POS_ADV': 1, 'FOLLOWING..."
33,"{'VERB': 1, 'FOLLOWING_POS_PROPN': 1, 'FOLLOWI..."
1328,"{'VERB': 1, 'FOLLOWING_POS_PUNCT': 1, 'FOLLOWI..."
1280,"{'VERB': 2, 'FOLLOWING_POS_DET': 1, 'FOLLOWING..."
1091,"{'VERB': 1, 'FOLLOWING_POS_PUNCT': 1, 'FOLLOWI..."
948,"{'VERB': 3, 'FOLLOWING_POS_PART': 1, 'FOLLOWIN..."
1137,"{'VERB': 1, 'FOLLOWING_POS_ADP': 1, 'FOLLOWING..."


Convert one feature dictionary to dataframe...

In [55]:
most_incorrect[['features']].iloc[1].values[0]

{'CHILD_DEP_DOBJ': 1,
 'CHILD_DEP_INTJ': 1,
 'CHILD_DEP_NPADVMOD': 1,
 'CHILD_DEP_PUNCT': 1,
 'CHILD_POSTAG_,': 1,
 'CHILD_POSTAG_NNP': 2,
 'CHILD_POSTAG_UH': 1,
 'CHILD_POS_INTJ': 1,
 'CHILD_POS_PROPN': 2,
 'CHILD_POS_PUNCT': 1,
 'FOLLOWING_POSTAG_NNP': 1,
 'FOLLOWING_POS_PROPN': 1,
 'PARENT_DEP_ROOT': 1,
 'PARENT_POSTAG_VB': 1,
 'PARENT_POS_VERB': 1,
 'VERB': 1}

In [56]:
f_dict = pd.DataFrame.from_dict(most_incorrect[['features']].iloc[1].values[0], orient='index').reset_index()
f_dict

,index,0
0,VERB,1
1,FOLLOWING_POS_PROPN,1
2,FOLLOWING_POSTAG_NNP,1
3,PARENT_DEP_ROOT,1
4,PARENT_POS_VERB,1
5,PARENT_POSTAG_VB,1
6,CHILD_DEP_NPADVMOD,1
7,CHILD_POS_PROPN,2
8,CHILD_POSTAG_NNP,2
9,CHILD_DEP_PUNCT,1


In [57]:
f_dict.columns = ['feature', 'count']
f_dict

,feature,count
0,VERB,1
1,FOLLOWING_POS_PROPN,1
2,FOLLOWING_POSTAG_NNP,1
3,PARENT_DEP_ROOT,1
4,PARENT_POS_VERB,1
5,PARENT_POSTAG_VB,1
6,CHILD_DEP_NPADVMOD,1
7,CHILD_POS_PROPN,2
8,CHILD_POSTAG_NNP,2
9,CHILD_DEP_PUNCT,1


Aggregating two feature dataframes into one by feature occurrence count...

In [58]:
pd.concat([f_dict, f_dict]).groupby('feature').sum().reset_index()

,feature,count
0,CHILD_DEP_DOBJ,2
1,CHILD_DEP_INTJ,2
2,CHILD_DEP_NPADVMOD,2
3,CHILD_DEP_PUNCT,2
4,"CHILD_POSTAG_,",2
5,CHILD_POSTAG_NNP,4
6,CHILD_POSTAG_UH,2
7,CHILD_POS_INTJ,2
8,CHILD_POS_PROPN,4
9,CHILD_POS_PUNCT,2


Putting it all together...

In [59]:
f_dicts = []
for i in range(len(most_incorrect)):
    f_dict = pd.DataFrame.from_dict(most_incorrect[['features']].iloc[i].values[0], orient='index').reset_index()
    f_dict.columns = ['feature', 'count']
    f_dicts.append(f_dict)
f_dicts

[               feature  count
 0                 VERB      1
 1   FOLLOWING_POS_NOUN      1
 2  FOLLOWING_POSTAG_NN      1
 3      PARENT_DEP_ROOT      1
 4     PARENT_POS_PROPN      1
 5    PARENT_POSTAG_NNP      1
 6       CHILD_DEP_DOBJ      1
 7       CHILD_POS_NOUN      1
 8      CHILD_POSTAG_NN      1,                  feature  count
 0                   VERB      1
 1    FOLLOWING_POS_PROPN      1
 2   FOLLOWING_POSTAG_NNP      1
 3        PARENT_DEP_ROOT      1
 4        PARENT_POS_VERB      1
 5       PARENT_POSTAG_VB      1
 6     CHILD_DEP_NPADVMOD      1
 7        CHILD_POS_PROPN      2
 8       CHILD_POSTAG_NNP      2
 9        CHILD_DEP_PUNCT      1
 10       CHILD_POS_PUNCT      1
 11        CHILD_POSTAG_,      1
 12        CHILD_DEP_INTJ      1
 13        CHILD_POS_INTJ      1
 14       CHILD_POSTAG_UH      1
 15        CHILD_DEP_DOBJ      1,                  feature  count
 0                   VERB      1
 1      FOLLOWING_POS_ADJ      1
 2  FOLLOWING_POSTAG_PRP$     

In [60]:
pd.concat(f_dicts).groupby('feature').sum().sort_values('count', ascending=False).reset_index()

,feature,count
0,VERB,27
1,PARENT_DEP_ROOT,22
2,PARENT_POS_VERB,19
3,CHILD_DEP_DOBJ,18
4,CHILD_POS_ADP,15
5,CHILD_POS_NOUN,15
6,CHILD_POSTAG_IN,15
7,PARENT_POSTAG_VB,14
8,CHILD_DEP_PREP,12
9,CHILD_POS_PUNCT,10


#### aggregate_features(preds_df, n=0)

In [61]:
def aggregate_features(preds_df, n=0):
    f_dicts = []
    for i in range(len(most_incorrect)):
        f_dict = pd.DataFrame.from_dict(preds_df[['features']].iloc[i].values[0], orient='index').reset_index()
        f_dict.columns = ['feature', 'count']
        f_dicts.append(f_dict)
    
    agg = pd.concat(f_dicts).groupby('feature').sum().sort_values('count', ascending=False).reset_index()
    
    if n <= 0:
        return agg
    else:
        return agg[:n]

In [62]:
aggregate_features(most_incorrect)

,feature,count
0,VERB,27
1,PARENT_DEP_ROOT,22
2,PARENT_POS_VERB,19
3,CHILD_DEP_DOBJ,18
4,CHILD_POS_ADP,15
5,CHILD_POS_NOUN,15
6,CHILD_POSTAG_IN,15
7,PARENT_POSTAG_VB,14
8,CHILD_DEP_PREP,12
9,CHILD_POS_PUNCT,10


In [63]:
aggregate_features(most_incorrect, 10)

,feature,count
0,VERB,27
1,PARENT_DEP_ROOT,22
2,PARENT_POS_VERB,19
3,CHILD_DEP_DOBJ,18
4,CHILD_POS_ADP,15
5,CHILD_POS_NOUN,15
6,CHILD_POSTAG_IN,15
7,PARENT_POSTAG_VB,14
8,CHILD_DEP_PREP,12
9,CHILD_POS_PUNCT,10


In [64]:
aggregate_features(most_correct, 10)

,feature,count
0,VERB,49
1,PARENT_POS_VERB,37
2,CHILD_DEP_NSUBJ,35
3,CHILD_POS_NOUN,35
4,PARENT_DEP_ROOT,34
5,CHILD_POSTAG_NN,28
6,PARENT_POSTAG_VBZ,28
7,CHILD_POS_PUNCT,24
8,CHILD_DEP_PUNCT,24
9,CHILD_POS_ADJ,16
